In [ ]:
#Imports 
import sys
sys.path.append('/Users/viktoriiaovsianik/Documents/Uni/04_WS2024/06_ADL/Code/ADL-WS-2024')
from scripts.train_cnn import train_model
from scripts.viz import display_images

from datasets.AIArtBench import AIArtbench
from datasets.dataset import Subset
from datasets.preprocessing import CustomDatasetPreprocessor
from torch.utils.data import DataLoader
from models.cnn import CNN_Net

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ovsvc (ovsvc-tu-wien). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/viktoriiaovsianik/.netrc


#### Import dataset

In [2]:
# Define your dataset path and preprocess the data
dataset_path = "/Users/viktoriiaovsianik/Documents/Uni/04_WS2024/06_ADL/Code/ADL-WS-2024/data"
preprocessor = CustomDatasetPreprocessor(dataset_path)
preprocessor.preprocess(fraction=0.2)

# Get dataset splits
train_data, validation_data, test_data = preprocessor.get_splits()

# Prepare the training dataset and DataLoader
train_dataset = AIArtbench(dataframe=train_data, subset=Subset.TRAINING, transform=None)

In [3]:
#Plot images
train_dataset.plot_images(label_type='human', k=5)
train_dataset.plot_images(label_type='AI', k=5)

# Extract images
#train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# Get unique class names
#classes = train_data['label'].unique()
# Display a few images from the training data
#display_images(data_loader=train_data_loader, classes=classes, num_images=8, save_path="train_images_grid.png")

AttributeError: 'AIArtbench_opt' object has no attribute 'plot_images'

#### Train baseline

In [4]:
model = CNN_Net()

In [5]:
config = {
    "dataset_path": dataset_path,  # Modify as needed
    "fraction": 0.1,
    "learning_rate": 0.001,
    "scheduler_gamma": 0.9,
    "epochs": 5,
    "batch_size": 64,
    "val_frequency": 3,
    "model_save_dir": "saved_models",
    "debug_mode": "True",
    "model": model
}

train_model(config)

CUDA (GPU) is not available. Training on CPU.
Preprocessing dataset...
Train dataset length: 1394
Validation dataset length: 156
Preparing data transforms...
Loading training and validation data...
Check loaded data: 1394
Setting up the model and optimizer...
Model: CNN_Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=20, bias=True)
)


Training with batch size: 64
Epoch 0/5:
--- Training epoch 0 ---


Train:   5%|▍         | 1/22 [00:00<00:06,  3.47it/s]

Batch 0, Loss: 3.0111677646636963


Train:  50%|█████     | 11/22 [00:02<00:02,  5.00it/s]

...

Train: 100%|██████████| 22/22 [00:05<00:00,  4.35it/s]

Batch 20, Loss: 2.9942219257354736


Train: 100%|██████████| 22/22 [00:05<00:00,  4.09it/s]


Epoch 4 Training Loss: 3.00195654085069
Training Metrics: Overall Accuracy: 0.0789
 Per-Class Accuracy:
Accuracy for class AI_ukiyo-e: 0.00
Accuracy for class AI_impressionism: 0.00
Accuracy for class human_baroque: 0.00
Accuracy for class AI_surrealism: 0.00
Accuracy for class AI_romanticism: 0.00
Accuracy for class AI_post_impressionism: 0.00
Accuracy for class human_realism: 0.00
Accuracy for class human_expressionism: 0.00
Accuracy for class human_romanticism: 0.00
Accuracy for class human_impressionism: 1.00
Accuracy for class AI_art_nouveau: 0.00
Accuracy for class AI_realism: 0.00
Accuracy for class AI_renaissance: 0.00
Accuracy for class AI_expressionism: 0.00
Accuracy for class human_art_nouveau: 0.00
Accuracy for class human_ukiyo_e: 0.00
Accuracy for class human_post_impressionism: 0.00
Accuracy for class human_surrealism: 0.00
Accuracy for class AI_baroque: 0.00
Accuracy for class human_renaissance: 0.00


epoch,▁▃▅▆█
train/acc,▁▅███
train/loss,█▆▄▂▁
val/acc,▁█
val/loss,█▁
epoch,4
train/acc,0.07891
train/loss,3.00196
val/acc,0.05128
val/loss,3.01114


In [12]:
from models.simple_cnn import Simple_CNN

model_simple = Simple_CNN()

config_simple = {
    "dataset_path": dataset_path,  # Modify as needed
    "fraction": 0.7,
    "learning_rate": 0.001,
    "scheduler_gamma": 0.9,
    "epochs": 5,
    "batch_size": 64,
    "val_frequency": 3,
    "model_save_dir": "saved_models",
    "debug_mode": "True",
    "model": model_simple
}

train_model(config_simple)

CUDA (GPU) is not available. Training on CPU.
Preprocessing dataset...
Train dataset length: 53508
Validation dataset length: 5949
Preparing data transforms...
Loading training and validation data...
Check loaded data: 53508
Setting up the model and optimizer...
Model: Simple_CNN(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=1024, out_features=20, bias=True)
)
Training with batch size: 64
Epoch 0/5:
--- Training epoch 0 ---


Train:   0%|          | 1/837 [00:00<04:59,  2.79it/s]

Batch 0, Loss: 2.9966914653778076


Train:   1%|▏         | 11/837 [00:03<05:10,  2.66it/s]

Batch 10, Loss: 3.0046849250793457


Train:   2%|▏         | 13/837 [00:04<05:00,  2.74it/s]


KeyboardInterrupt: 